In [1]:
ours2orig = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 254, 41, 42, 43, 44, 45,
46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,
65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 85,
86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103,
104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118,
119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 136, 138, 132,
133, 134, 135, 220, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 185, 186,
187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,
217, 218, 219, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
233, 137, 234, 235, 236, 237, 238, 239, 240, 241, 250, 251, 255, 256, 257,
258, 259, 260, 261, 262, 263, 242, 243, 244, 245, 0, 1, 2, 3, 4, 5, 6, 7, 8,
9, 10, 11, 83, 84, 131, 139, 140, 141, 181, 182, 183, 184, 246, 247, 248,
249, 252, 253]

def vec2mat(v):
    a,b = np.triu_indices(264,1)
    m = np.zeros((264,264))
    m[a,b] = v
    return m+m.T

def remap(fc, roimap=ours2orig):
    fc = fc[roimap,:]
    fc = fc[:,roimap]
    return fc

print('Complete')

Complete


In [2]:
import pickle
import numpy as np

# s1.0 = male probably

bsnipdir = '/home/anton/Documents/Tulane/Research/data-sav/anton/cohorts/BSNIP/'
demo = pickle.load(open(f'{bsnipdir}/demographics.pkl', 'rb'))

fc = []
pc = []
race = []
sex = []
age = []
sz = []
subids = []

a,b = np.triu_indices(264,1)

for sub in demo['Age_cal']:
    diag = demo['DXGROUP_1'][sub]
    if diag not in ['SZP', 'NC']:
        continue
    ra = demo['Race'][sub]
    if ra not in ['AA', 'CA']:
        continue
    se = demo['sex'][sub]
    diag = diag == 'SZP'
    ra = ra == 'AA'
    se = se == 's1.0'
    ag = demo['Age_cal'][sub]
    p = np.load(f'{bsnipdir}/fc/{sub}_task-unk_fc.npy')
    p = remap(vec2mat(p))[a,b]
    age.append(ag)
    sex.append(se)
    race.append(ra)
    sz.append(diag)
    fc.append(p)
    subids.append(sub)

fc = np.stack(fc)
race = np.array(race).astype('int')
sex = np.array(sex).astype('int')
sz = np.array(sz).astype('int')
age = np.array(age)

print(max(age))
print(min(age))
print(np.mean(age))
print(np.std(age))
# age = (age-np.mean(age))/np.std(age)

print([a.shape for a in [fc, sz, race, sex, age]])

64.0
15.0
36.92098765432099
12.335582538145621
[(405, 34716), (405,), (405,), (405,), (405,)]


In [15]:
from scipy.stats import ttest_ind
print(np.sum((sz == 1) * (sex == 1)))
print(np.sum((sz == 1) * (sex == 0)))
res = ttest_ind((sz == 1) * (sex == 1), (sz == 1) * (sex == 0))
print(res.pvalue)

130
55
2.2099675919612228e-10


In [19]:
from nilearn.connectome import ConnectivityMeasure

cm = ConnectivityMeasure(kind='partial correlation')

tsfile = pickle.load(open('/home/anton/Documents/Tulane/Research/data-sav/anton/cohorts/BSNIP/bsnip_rc_ts_centered.pkl', 'rb'))
a,b = np.triu_indices(264,1)

for i,sub in enumerate(subids):
    subold = sub.replace('.','_')
    try:
        pc = cm.fit_transform(np.expand_dims(tsfile[subold][ours2orig,:].T, 0))[0]
        np.save(f'/home/anton/Documents/Tulane/Research/ImageNomer/data/BSNIP/pc/{sub}_task-unk_pc.npy', pc[a,b])
    except:
        pass
    if i % 50 == 0:
        print(i)

print('Done')

0
50
100
150
200
250
300
350
400
Done


In [3]:
pc = []

for sub in subids:
    p = np.load(f'/home/anton/Documents/Tulane/Research/ImageNomer/data/BSNIP/pc/{sub}_task-unk_pc.npy')
    pc.append(p)

pc = np.stack(pc)

print(pc.shape)

(405, 34716)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import random

class VAE(nn.Module):
    def __init__(self, fcd, ld):
        super(VAE, self).__init__()
        self.fcd = fcd
        self.ld = ld
        self.enc1 = nn.Linear(fcd, 1000).float().cuda()
        self.enc2 = nn.Linear(1000, ld).float().cuda()
        self.dec1 = nn.Linear(ld+4, 1000).float().cuda()
        self.dec2 = nn.Linear(1000, fcd).float().cuda()

    def enc(self, x):
        x = F.relu(self.enc1(x))
        z = self.enc2(x)
        return z

    def gen(self, n):
        return torch.randn(n, self.ld).float().cuda()/(10**0.5)
    
    def dec(self, z, age, sex, race, sz):
        z = torch.cat([z, age.unsqueeze(1), sex.unsqueeze(1), race.unsqueeze(1), sz.unsqueeze(1)], dim=1)
        x = F.relu(self.dec1(z))
        x = self.dec2(x)
        return x

def rmse(a, b, mean=torch.mean):
    return mean((a-b)**2)**0.5

def pretty(x):
    return f'{round(float(x), 4)}'

vae = VAE(34716, 30)
vae.load_state_dict(torch.load('/home/anton/Documents/Tulane/Research/ImageNomer/data/BSNIP/vae_1000_z30_cov4.torch'))
vae.eval()

with torch.no_grad():
    z = vae.enc(torch.from_numpy(fc).float().cuda())
    age_t = torch.from_numpy(age).float().cuda()
    sex_t = torch.from_numpy(sex).float().cuda()
    race_t = torch.from_numpy(race).float().cuda()
    sz_t = torch.from_numpy(sz).float().cuda()
    ffc = vae.dec(z, age_t, sex_t, race_t, sz_t).detach().cpu().numpy()

print('Done')

Done


In [24]:
from nilearn.connectome import ConnectivityMeasure

cm = ConnectivityMeasure(kind='partial correlation')

xpc = []

def pc_from_fc(fc):
    diag = np.random.uniform(low=10,high=20,size=(264))
    d1 = np.expand_dims(diag,0)
    d2 = np.expand_dims(diag,1)
    I = vec2mat(fc)
    l, v = np.linalg.eig(I)
    l[l<0] = 1e-3
    I = v@np.diag(l)@v.T
    # I = 0.99*I+0.01*np.eye(264)
    I = I*d1*d2
    L = np.linalg.cholesky(I)
    x = np.random.normal(loc=0, scale=1, size=(264,200))
    Lx = L@x
    pc = cm.fit_transform([Lx.T])[0]
    a,b = np.triu_indices(264,1)
    return pc[a,b]

for i in range(len(ffc)):
    xpc.append(pc_from_fc(ffc[i]))
    if i % 20 == 0:
        print(i)

xpc = np.stack(xpc)

print('Done')
print(xpc.shape)

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
Done
(405, 34716)


In [11]:
from sklearn.linear_model import Ridge

def rmse(a,b,mean=np.mean):
    return mean((a-b)**2)**0.5

reg = Ridge(alpha=1).fit(ffc, age)
yhat = reg.predict(fc)
err = rmse(yhat, age)
print(err)
null = rmse(0, age)
print(null)

0.6051844124327149
1.0


In [29]:
from sklearn.linear_model import LogisticRegression

reg = LogisticRegression(C=10000).fit(xpc, race)
yhat = reg.predict(pc)
acc = np.mean(yhat == race)
print(acc)
null = np.mean(sz)
if null < 0.5:
    null = 1-null
print(null)

0.9061728395061729
0.5432098765432098
